# Disease Norm

In this example, we'll be writing an application to extract *mentions of* diseases from Pubmed abstracts, using annotations from the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial, which has 5 parts, walks through the process of constructing a model to classify _candidate_ disease mentions as either true (i.e., that it is truly a mention of a disease) or false.

## Plan of action:

Two types of LFs:
1. TYPE I: Leveraging sources of WS (e.g. DS)
2. TYPE II: Expressing heuristics (e.g. magnifying user effort)

TYPE I:
- Need to break up MESH into subtrees and have each one be an LF!
- Need to provide negative signal

TYPE II:
- Conduct "simulated expert" experiment: go through, label examples, write LFs- what is the effective multiplier over binary labeling??
    * E.g. "renal failure"; add {"renal" -> "kidney"} to synonym map

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass

Disease = candidate_subclass('Disease', ['disease'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from snorkel.models import CandidateSet

train = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Training Candidates').one()
print len(train)
dev = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Development Candidates').one()
print len(dev)

30067
29853


In [4]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

L_gold_train = label_manager.load(session, train, "CDR Training Label Set")
print L_gold_train.shape
L_gold_dev = label_manager.load(session, dev, "CDR Development Label Set")
print L_gold_dev.shape

(30067, 1)
(29853, 1)


# Writing some multinomial LFs

## TYPE I LF: Subsets of MESH dictionary

In [5]:
from cPickle import load
from utils import load_mesh_raw

MESH_to_CID = load(open('MESH_to_CID.pkl', 'rb'))
mesh_entries = load_mesh_raw('data/desc2017.xml')

Loaded 28472 entries


## MESH exact match

In [6]:
mesh_tree = defaultdict(list)
for entry in mesh_entries:
    mid, tree_nums, terms = entry
    for tn in tree_nums:
        path = [tn[0]] + tn[1:].split(".")
        for term in terms:
            mesh_tree[term].append((mid, path))

### Augmenting MESH with UMLS

In [50]:
mid_to_paths = defaultdict(list)
for entry in mesh_entries:
    mid, tree_nums, terms = entry
    for tn in tree_nums:
        path = [tn[0]] + tn[1:].split(".")
        mid_to_paths[mid].append(path)

In [51]:
len(mesh_tree)

231583

In [53]:
with open('cui2mesh.tsv', 'rb') as f:
    for line in f:
        term, cui, mid = line.rstrip('\n').split('\t')
        for path in mid_to_paths[mid]:
            x = (mid, path)
            if x not in mesh_tree[term]:
                mesh_tree[term].append(x)

In [54]:
len(mesh_tree)

478507

In [55]:
POS_DEPTH = 2
NEG_DEPTH = 1

def LFG_MESH_exact(c):
    p = c.disease.get_span().lower()
    if p in mesh_tree:
        seen = set()
        for mid, path in mesh_tree[p]:
            value = MESH_to_CID[mid] if path[0] in ['C', 'F'] else -1
            key   = "_".join(path[:POS_DEPTH]) if value > 0 else "_".join(path[:NEG_DEPTH])
            if key not in seen:
                seen.add(key)
                yield key, value

In [56]:
%time L_train = label_manager.create(session, train, 'LF Training Labels -- MESH + UMLS Exact + Cosine', f=LFG_MESH_exact)
L_train

[========================================] 100%

Loading sparse Label matrix...
CPU times: user 1min 28s, sys: 15.9 s, total: 1min 44s
Wall time: 1min 33s


<30067x41 sparse matrix of type '<type 'numpy.float64'>'
	with 9454 stored elements in Compressed Sparse Row format>

In [ ]:
# LOAD if already computed
L_train = label_manager.load(session, train, 'LF Training Labels -- MESH Exact + Cosine')
L_train

## MESH TF-IDF cosine match

In [59]:
%%time
from entity_norm import CanonDictVectorizer

# Compile all terms into one dictionary
all_diseases = {}
for term, entries in mesh_tree.iteritems():
    if len(entries) > 0:
        mid, path = entries[0]  # Hack: should take the most frequent?
        all_diseases[term] = mid

# Create a vectorizer based around this 
cd_vectorizer = CanonDictVectorizer(all_diseases, other_phrases=[])

# Vectorize the dictionary
disease_phrases = []
disease_cids    = []
for term, mid in all_diseases.iteritems():
    disease_phrases.append(term)
    disease_cids.append(mid)   
D  = cd_vectorizer.vectorize_phrases(disease_phrases)
Dt = D.T
Dt

|V| = 49753
CPU times: user 2min 54s, sys: 1.02 s, total: 2min 55s
Wall time: 2min 55s


In [146]:
Dt

<49754x478503 sparse matrix of type '<type 'numpy.float64'>'
	with 1202586 stored elements in Compressed Sparse Column format>

In [60]:
mesh_tree_index = []
for dp in disease_phrases:
    mesh_tree_index.append(mesh_tree[dp])

In [151]:
THRESH = 0.75

def LFG_MESH_cosine(c):
    
    # Vectorize the phrase
    p  = c.disease.get_span().lower()
    cx = cd_vectorizer.vectorize_phrases([p])
    m  = cx * Dt
    
    # Keep track of the highest-score match so far _for each LF_
    highest_score = defaultdict(float)
    
    # Iterate over non-zero dictionary term matches > THRESH
    for i, j in zip(*m.nonzero()):
        s = m[i,j]
        if s > THRESH:
            for entry in mesh_tree_index[j]:
                mid, path = entry
                
                # We define each LF by a tree path code
                key = "_".join(path[:2]) + "_cosine"
                
                # Only yield this value if higher than highest current emitted
                # Note: This will just update the current value in the DB
                if s > highest_score[key]:
                    highest_score[key] = s
                    yield key, MESH_to_CID[mid] if path[0] in ['C', 'F'] else -1

In [62]:
%time L_train = label_manager.update(session, train, 'LF Training Labels -- MESH + UMLS Exact + Cosine', True, LFG_MESH_cosine)
L_train

[========================================] 100%

Loading sparse Label matrix...
CPU times: user 54min 15s, sys: 1min 31s, total: 55min 46s
Wall time: 55min 23s


<30067x155 sparse matrix of type '<type 'numpy.float64'>'
	with 55855 stored elements in Compressed Sparse Row format>

In [ ]:
# LOAD if already computed
L_train = label_manager.load(session, train, 'LF Training Labels -- MESH Exact + Cosine')
L_train

# Running gen. model

In [9]:
from scipy.sparse import lil_matrix

def binarize_LF_matrix(X):
    X_b = lil_matrix(X.shape)
    for i, j in zip(*X.nonzero()):
        X_b[i,j] = np.sign(X[i,j])
    return X_b.tocsr()

In [14]:
def get_score(predicted, gold):
    tp = 0
    pp = 0
    p  = 0
    for i in range(gold.shape[0]):
        if gold[i] > 0:
            p += 1
        
        if predicted[i] == 1:
            pp += 1
            if gold[i] > 0:
                tp += 1
    
    prec   = tp / float(pp)
    recall = tp / float(p)
    f1     = (2*prec*recall) / (prec+recall)
    print "P :\t", prec
    print "R :\t", recall
    print "F1:\t", f1

In [63]:
L_train_b = binarize_LF_matrix(L_train)
L_train_b

<30067x155 sparse matrix of type '<type 'numpy.float64'>'
	with 55855 stored elements in Compressed Sparse Row format>

In [89]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
%time gen_model.train(L_train_b, n_iter=10000, verbose=False)

Training marginals (!= 0.5):	30067
Features:			155
CPU times: user 18.9 s, sys: 57.8 ms, total: 18.9 s
Wall time: 18.9 s


In [90]:
yp = gen_model.predict(L_train_b)
get_score(yp, L_gold_train)

P :	0.672828740844
R :	0.496780839557
F1:	0.571555555556


In [28]:
yp = gen_model.predict(L_train_b)
get_score(yp, L_gold_train)

P :	0.737365368683
R :	0.458408447077
F1:	0.565348578688


In [17]:
yp = gen_model.predict(L_train_b)
get_score(yp, L_gold_train)

P :	0.798275862069
R :	0.357713108421
F1:	0.49404232616


In [ ]:
from snorkel.learning import odds_to_prob

L_dev.lf_stats(labels=L_gold_dev, est_accs=odds_to_prob(gen_model.w))

In [ ]:
yp = gen_model.predict(L_dev_b)
get_score(yp, L_gold_dev)

In [ ]:
from snorkel.utils import matrix_accuracy

eaccs = matrix_accuracy(L_dev, L_gold_dev)

In [ ]:
from snorkel.learning import odds_to_prob

L_dev.lf_stats(labels=L_gold_dev, est_accs=odds_to_prob(gen_model.w))

# Error analysis

In [84]:
len(train)

30067

In [93]:
fps = []
fns = []
for i,c in enumerate(train):
    if L_gold_train[i] < 0 and yp[i] > 0:
        fps.append(c)
    elif L_gold_train[i] > 0 and yp[i] <= 0:
        fns.append(c)
print "FPs:", len(fps)
print "FNs:", len(fns)

FPs: 938
FNs: 1954


In [94]:
from random import shuffle
shuffle(fps)
shuffle(fns)

In [110]:
# Index the gold MESH IDs
CID_to_MESH = {}
for mid, cid in MESH_to_CID.iteritems():
    CID_to_MESH[cid] = mid
    
mesh_label_by_candidate_id = {}
for i, c in enumerate(train):
    l = int(L_gold_train[i,0])
    mesh_label_by_candidate_id[c.id] = CID_to_MESH[l] if l > 0 else l

In [114]:
mesh_to_terms = defaultdict(set)
for term, entries in mesh_tree.iteritems():
    for entry in entries:
        mid, path = entry
        mesh_to_terms[mid].add(term)

In [175]:
from snorkel.viewer import SentenceNgramViewer

sv = SentenceNgramViewer(fns[:100], session)

<IPython.core.display.Javascript object>

In [176]:
sv

# Experiments to run:
* Clean up!
* Partial matches (first k words)
* TF-IDF at various thresholds
* Split dictionary more!
* Remove JJ?
* **NEED TO CORRECT FOR GOLD ANNOTATIONS NOT IN OUR CANDIDATE SET!!!**

In [177]:
c = sv.get_selected()
c

Disease(Span("advanced encephalopathy", parent=1242, chars=[17,39], words=[3,4]))

In [178]:
mid = mesh_label_by_candidate_id[sv.get_selected().id]
print mid
mesh_to_terms[mid]

D001927


{'BRAIN DIS',
 'Brain Disease',
 'Brain Diseases',
 'Brain Disorder',
 'Brain Disorders',
 'CNS DIS INTRACRANIAL',
 'CNS Disorder, Intracranial',
 'CNS Disorders, Intracranial',
 'CNS INTRACRANIAL DIS',
 'Central Nervous System Disorders, Intracranial',
 'Central Nervous System Intracranial Disorders',
 'ENCEPHALON DIS',
 'Encephalon Disease',
 'Encephalon Diseases',
 'INTRACRANIAL CNS DIS',
 'Intracranial CNS Disorder',
 'Intracranial CNS Disorders',
 'Intracranial Central Nervous System Disorders',
 'brain disease',
 'brain diseases',
 'brain disorder',
 'brain disorders',
 'central nervous system disorders, intracranial',
 'central nervous system intracranial disorders',
 'cns disorder, intracranial',
 'cns disorders, intracranial',
 'encephalon disease',
 'encephalon diseases',
 'encephalopathies',
 'encephalopathy',
 'intracranial central nervous system disorders',
 'intracranial cns disorder',
 'intracranial cns disorders'}

In [179]:
list(LFG_MESH_exact(c))

[]

In [180]:
list(LFG_MESH_cosine(c))

[]

### Notes:
* Looking at FNs:

## Automatically Creating Features
Recall that our goal is to distinguish between true and false mentions of chemical-disease relations. To train a model for this task, we first embed our `ChemicalDisease` candidates in a feature space.

In [29]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()

We can create a new feature set:

In [40]:
%time F_train = feature_manager.create(session, train, 'Train Features 3')

[========================================] 100%

Loading sparse Feature matrix...
CPU times: user 18min 58s, sys: 23.3 s, total: 19min 22s
Wall time: 19min 8s


**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_train = feature_manager.load(session, train, 'Train Features')

Note that the returned matrix is a special subclass of the `scipy.sparse.csr_matrix` class, with some special features which we demonstrate below:

In [41]:
F_train

<30067x72203 sparse matrix of type '<type 'numpy.float64'>'
	with 1230157 stored elements in Compressed Sparse Row format>

In [47]:
%time F_dev = feature_manager.update(session, dev, 'Train Features 3', False)

[========================================] 100%

Loading sparse Feature matrix...
CPU times: user 11min 2s, sys: 30.3 s, total: 11min 32s
Wall time: 11min 19s


In [ ]:
F_train.get_candidate(0)

In [ ]:
F_train.get_key(0)

In [81]:
from snorkel.learning import LogReg

train_marginals = gen_model.marginals(L_train_b)

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=5000, rate=1e-3)

Training marginals (!= 0.5):	17361
Features:			72203
Using gradient descent...
	Learning epoch = 0	Step size = 0.001
	Loss = 12033.728202	Gradient magnitude = 8418.569490
	Learning epoch = 100	Step size = 0.000904792147114
	Loss = 3574.614580	Gradient magnitude = 1667.630374
	Learning epoch = 200	Step size = 0.000818648829479
	Loss = 2788.945606	Gradient magnitude = 462.129656
	Learning epoch = 300	Step size = 0.000740707032156
	Loss = 2609.218558	Gradient magnitude = 37.593165
	Learning epoch = 400	Step size = 0.000670185906007
	Loss = 2525.500892	Gradient magnitude = 31.789598
	Learning epoch = 500	Step size = 0.000606378944861
	Loss = 2469.144214	Gradient magnitude = 27.994361
	Learning epoch = 600	Step size = 0.000548646907485
	Loss = 2428.639760	Gradient magnitude = 25.295902
	Learning epoch = 700	Step size = 0.000496411413431
	Loss = 2398.222866	Gradient magnitude = 23.281157
	Learning epoch = 800	Step size = 0.00044914914861
	Loss = 2374.639587	Gradient magnitude = 21.724859
	Le

In [82]:
yp = disc_model.predict(F_train)
get_score(yp, L_gold_train)

P :	0.669799008092
R :	0.66082925573
F1:	0.665283899404


In [44]:
yp = disc_model.predict(F_train)
get_score(yp, L_gold_train)

P :	0.728406610539
R :	0.60159670358
F1:	0.658956276446


In [83]:
yp = disc_model.predict(F_dev)
get_score(yp, L_gold_dev)

P :	0.692197360872
R :	0.603853853854
F1:	0.645014701951


In [48]:
yp = disc_model.predict(F_dev)
get_score(yp, L_gold_dev)

P :	0.754307374225
R :	0.547797797798
F1:	0.634676717889


# TODO:

0. _DONE: Add negative labels to candidates..._
1. _DONE: Get empirical LF accs up and running..._
2. _DONE: Binarize LFs + run in binary gen model_
0. _DONE: Add TF-IDF matching LFs_
3. _DONE: Add in simple DDLIB + WS feats from new-features -> run disc. model_
2. **Conduct TYPE II "experiment"!**

## Creating Labeling Functions
Labeling functions are a core tool of data programming. They are heuristic functions that aim to classify candidates correctly. Their outputs will be automatically combined and denoised to estimate the probabilities of training labels for the training data.

In [ ]:
import re
from lf_terms import *
from snorkel.lf_helpers import get_left_tokens, get_right_tokens

We also load some publicly-available biomedical dictionaries, which we will leverage in some of our LFs below as a source of weak supervision:

In [ ]:
from utils import *

umls_dict              = load_umls_dictionary()
chemicals              = load_chemdner_dictionary()
abbrv2text, text2abbrv = load_specialist_abbreviations()

#### Document-Level Labeling Functions
We start with some labeling functions that label candidates based on document-level features.

In [ ]:
from snorkel.lf_helpers import get_doc_candidate_spans

def LF_undefined_abbreviation(c):
    '''Candidate is a known abbreviation, but no corresponding full name in document'''
    doc_spans = get_doc_candidate_spans(c)
    phrase = c[0].get_span().lower()
    mentions = set([s.get_span().lower() for s in doc_spans])
    if len(phrase) > 1 and phrase in abbrv2text and not set(abbrv2text[phrase].keys()).intersection(mentions):
        return -1
    return 0

#### Sentence-Level Labeling Functions
We also include some labeling functions that label candidates based on sentence-level features.

In [ ]:
from snorkel.lf_helpers import get_sent_candidate_spans

def LF_contiguous_mentions(c):
    '''Contiguous candidates are likely wrong'''
    neighbor_spans = get_sent_candidate_spans(c)
    start, end = c[0].get_word_start(), c[0].get_word_end()
    for s in neighbor_spans:
        if s.get_word_end() + 1 == start or s.get_word_start() - 1 == end:
            return -1
    return 0

#### Mention-Level Labeling Functions
We now define a number of labeling functions that label candidates based on attributes related to the mention.

In [ ]:
from snorkel.lf_helpers import get_left_tokens, get_right_tokens

def LF_tumors_growths(c):
    phrase = " ".join(c[0].get_attrib_tokens('lemmas'))
    return 1 if re.search("^(\w* ){0,2}(['] )*(tumor|tumour|polyp|pilomatricoma|cyst|lipoma)$", phrase) else 0

def LF_cancer(c):
    '''<TYPE> cancer'''
    phrase = " ".join(c[0].get_attrib_tokens('lemmas'))
    return 1 if re.search("\w* cancer",phrase) else 0

def LF_disease_syndrome(c):
    '''<TYPE> disease or <TYPE> syndrome'''
    phrase = " ".join(c[0].get_attrib_tokens('lemmas'))
    return 1 if re.search("\w* (disease|syndrome)+",phrase) else 0

def LF_indicators(c):
    '''Indicator words'''
    return 1 if " ".join(c[0].get_attrib_tokens()).lower() in indicators else 0

def LF_common_disease(c):
    '''Common disease'''
    return 1 if " ".join(c[0].get_attrib_tokens()).lower() in common_disease else 0

*For a few more examples of LFs of this style that we'll use, see [Disease_Tagging_Tutorial_LFs.py](Disease_Tagging_Tutorial_LFs.py).*

#### Dictionary Labeling Functions
We can use existing dictionaries for distant supervision.

In [ ]:
def LF_SNOWMED_CT_sign_or_symptom(c):
    return 1 if c[0].get_span() in umls_dict["snomedct"]["sign_or_symptom"] else 0

def LF_SNOWMED_CT_disease_or_syndrome(c):
    return 1 if c[0].get_span() in umls_dict["snomedct"]["disease_or_syndrome"] else 0

def LF_MESH_disease_or_syndrome(c):
    return 1 if c[0].get_span() in umls_dict["mesh"]["disease_or_syndrome"] else 0

def LF_MESH_sign_or_symptom(c):
    return 1 if c[0].get_span() in umls_dict["mesh"]["sign_or_symptom"] else 0

#### Negative Labeling Functions
When writing labeling functions, it is important to provide negative supervision in addition to positive supervision.

In [ ]:
def LF_organs(c):
    phrase = " ".join(c[0].get_attrib_tokens()).lower()
    return -1 if phrase in organs else 0      

def LF_chemical_name(c):
    phrase = " ".join(c[0].get_attrib_tokens())
    return -1 if phrase in chemicals and not phrase.isupper() else 0

def LF_bodysym(c):
    phrase = " ".join(c[0].get_attrib_tokens()).lower()
    return -1 if phrase in bodysym else 0  

def LF_protein_chemical_abbrv(c):
    '''Gene/protein/chemical name'''
    lemma = " ".join(c[0].get_attrib_tokens('lemmas'))
    return -1 if re.search("\d+",lemma) else 0

def LF_base_pair_seq(c): 
    lemma = " ".join(c[0].get_attrib_tokens('lemmas'))
    return -1 if re.search("^[GACT]{2,}$",lemma) else 0

*For a few more examples of LFs of this style that we'll use, see [Disease_Tagging_Tutorial_LFs.py](Disease_Tagging_Tutorial_LFs.py).*

We maintain a list of all LFs for convenience.

In [ ]:
from Disease_Tagging_Tutorial_LFs import *

LFs_doc = [LF_undefined_abbreviation]

LFs_sent = [LF_contiguous_mentions]

LFs_mention = [LF_tumors_growths,
               LF_cancer,
               LF_disease_syndrome,
               LF_indicators,
               LF_common_disease,
               LF_common_disease_acronyms,
               LF_deficiency_of,
               LF_positive_indicator,
               LF_left_positive_argument,
               LF_right_negative_argument,
               LF_medical_afixes,
               LF_adj_diseases
              ]

LFs_dicts =  [LF_SNOWMED_CT_sign_or_symptom,
              LF_SNOWMED_CT_disease_or_syndrome,
              LF_MESH_disease_or_syndrome,
              LF_MESH_sign_or_symptom
            ]

LFs_false = [LF_chemical_name,
             LF_organs,
             LF_bodysym,
             LF_protein_chemical_abbrv,
             LF_base_pair_seq,
             LF_too_vague,
             LF_neg_surfix,
             LF_non_common_disease,
             LF_non_disease_acronyms,
             LF_pos_in,
             LF_gene_chromosome_link,
             LF_right_window_incomplete,
             LF_negative_indicator
            ]

## Applying Labeling Functions

First we construct a `CandidateLabeler`.

In [ ]:
from snorkel.annotations import LabelManager

label_manager = LabelManager()

Next we run the `CandidateLabeler` to to apply the labeling functions to the training `CandidateSet`.  We'll start with some of our labeling functions:

In [ ]:
LFs = LFs_mention + LFs_dicts + LFs_false
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

**OR** load if we've already created:

In [ ]:
%time L_train = label_manager.load(session, train, 'LF Labels')
L_train

We can also add or rerun a single labeling function (or more!) with the below command. Note that we set the argument `expand_key_set` to `True` to indicate that the set of matrix columns should be allowed to expand:

In [ ]:
LFs_2   = LFs_doc + LFs_sent
L_train = label_manager.update(session, train, 'LF Labels', True, f=LFs_2)
L_train

We can view statistics about the resulting label matrix:

In [ ]:
L_train.lf_stats()

## Fitting the Generative Model
We estimate the accuracies of the labeling functions without supervision. Specifically, we estimate the parameters of a `NaiveBayes` generative model.

In [ ]:
from snorkel.learning import GenerativeModel

gen_model = GenerativeModel()
gen_model.train(L_train)

In [ ]:
gen_model.save(session, 'Generative Params')

We now apply the generative model to the training candidates.

In [ ]:
train_marginals = gen_model.marginals(L_train)

## Training the Discriminative Model
We use the estimated probabilites to train a discriminative model that classifies each `Candidate` as a true or false mention.

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=5000, rate=1e-3)

In [ ]:
disc_model.w.shape

In [ ]:
%time disc_model.save(session, "Discriminative Params")

## Evaluating on the Development `CandidateSet`

First, we create features for the development set.

Note that we use the training features feature set, because those are the only features for which we have learned parameters. Features that were not encountered during training, e.g., a token that does not appear in the training set, are ignored, because we do not have any information about them.

To do so with the `FeatureManager`, we call update with the new `CandidateSet`, the name of the training `AnnotationKeySet`, and the value `False` for the parameter `extend_key_set` to indicate that the `AnnotationKeySet` should not be expanded with new `Feature` keys encountered during processing.

In [ ]:
%time F_dev = feature_manager.update(session, dev, 'Train Features', False)

**OR** if we've already created one, we can simply load as follows:

In [ ]:
%time F_dev = feature_manager.load(session, dev, 'Train Features')

Next, we load the development set labels and gold candidates we made in Part III.

In [ ]:
L_gold_dev = label_manager.load(session, dev, "CDR Development Labels -- Gold")

In [ ]:
gold_dev_set = session.query(CandidateSet).filter(CandidateSet.name == 'CDR Development Candidates -- Gold').one()

Now we can evaluate the discriminative model on the development set.

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_gold_dev, gold_dev_set)

## Viewing Examples
After evaluating on the development `CandidateSet`, the labeling functions can be modified. Try changing the labeling functions to improve performance. You can view the true positives, false positives, true negatives, and false negatives using the `Viewer`.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

# NOTE: This if-then statement is only to avoid opening the viewer during automated testing of this notebook
# You should ignore this!
import os
if 'CI' not in os.environ:
    sv = SentenceNgramViewer(tp, session, annotator_name="Tutorial Part IV User")
else:
    sv = None

In [ ]:
sv

Next, in Part V, we will test our model on the test `CandidateSet`.